In [1]:
%load_ext autotime
import geopandas as gpd
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
from tqdm.auto import tqdm
from glob import glob
from shapely.geometry import LineString, Point
import folium
pd.set_option('display.max_columns', None)

In [7]:
new_shorelines = gpd.GeoDataFrame.from_features([{"type":"Feature","properties":{},"geometry":{"type":"LineString","coordinates":[[172.696667,-43.631602],[172.696238,-43.630484],[172.693748,-43.630484],[172.692289,-43.63036],[172.690659,-43.630981],[172.690916,-43.632099]]}}])
new_shorelines.crs = 4326
new_shorelines

,geometry
0,"LINESTRING (172.69667 -43.6316, 172.69624 -43...."


In [8]:
m = new_shorelines.explore(tiles="ESRI.WorldImagery")
gpd.GeoSeries(new_shorelines.geometry.apply(lambda line: Point(line.coords[0])), crs=new_shorelines.crs).explore(m=m, color="red", name="transect start")
m

In [9]:
shorelines = gpd.read_file("shorelines.geojson")
transects = gpd.read_file("transects_extended.geojson")
poly = gpd.read_file("polygons.geojson")

In [10]:
nz_shorelines = shorelines[shorelines.id.str.startswith("nzd")]
latest_siteid = nz_shorelines.id.max()
latest_siteid_int = int(latest_siteid[4:])
new_siteids = new_shorelines.index.to_series().apply(lambda i: f"nzd0{latest_siteid_int + i + 1}")
print(f"Latest siteid is {latest_siteid}, so the new sites will be {new_siteids}")
new_shorelines["id"] = new_siteids
new_shorelines

Latest siteid is nzd0579, so the new sites will be 0    nzd0580
dtype: object


,geometry,id
0,"LINESTRING (172.69667 -43.6316, 172.69624 -43....",nzd0580


In [11]:
pd.concat((shorelines, new_shorelines)).to_file("shorelines.geojson", driver="GeoJSON")

In [12]:
new_polys = gpd.GeoDataFrame({"id": new_siteids}, geometry=new_shorelines.to_crs(2193).buffer(100).minimum_rotated_rectangle().to_crs(4326), crs=4326)
new_polys.explore()

In [13]:
pd.concat((poly, new_polys)).to_file("polygons.geojson", driver="GeoJSON")

In [14]:
nz_transects = transects[transects.id.str.startswith("nzd")].to_crs(2193)
nz_transects

,id,site_id,orientation,along_dist,along_dist_norm,beach_slope,cil,ciu,trend,n_points,n_points_nonan,r2_score,mae,mse,rmse,intercept,ERODIBILITY,geometry
95050,nzd0001-0000,nzd0001,359.037136,3197.737936,1.000000,0.100,0.0636,0.2000,-0.048007,194.0,152.0,0.000243,19.220344,557.970123,23.621391,324.525695,None,"LINESTRING (1596662.375 6190263.089, 1596646.6..."
95051,nzd0001-0001,nzd0001,359.037136,3097.737936,0.968728,0.105,0.0695,0.2000,-0.193115,194.0,166.0,0.004693,17.523406,479.673721,21.901455,329.330311,None,"LINESTRING (1596744.903 6190264.499, 1596729.1..."
95052,nzd0001-0002,nzd0001,359.037136,2997.737936,0.937456,0.120,0.0757,0.2000,-0.280513,194.0,169.0,0.012589,15.782998,369.600504,19.224997,335.326896,None,"LINESTRING (1596827.431 6190265.908, 1596811.6..."
95053,nzd0001-0003,nzd0001,359.037136,2897.737936,0.906184,0.065,0.0546,0.0759,-0.319370,194.0,171.0,0.020481,14.218994,297.709530,17.254261,343.807158,None,"LINESTRING (1596909.959 6190267.317, 1596894.2..."
95054,nzd0001-0004,nzd0001,359.037136,2797.737936,0.874912,0.065,0.0567,0.0788,-0.385889,194.0,172.0,0.032665,13.279365,274.091797,16.555718,356.283944,None,"LINESTRING (1596992.486 6190268.724, 1596976.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147476,nzd0579-0000,nzd0579,NaN,NaN,NaN,0.040,0.0381,0.0416,-0.281528,884.0,690.0,0.008935,18.056946,517.647642,22.751871,247.801300,None,"LINESTRING (1603963.61 5467039.521, 1604360.95..."
147477,nzd0579-0001,nzd0579,NaN,NaN,NaN,0.160,0.1280,0.2000,0.189630,884.0,689.0,0.019535,7.332523,103.003893,10.149083,222.459728,None,"LINESTRING (1603955.09 5467142.194, 1604355.09..."
147478,nzd0579-0002,nzd0579,NaN,NaN,NaN,0.200,0.1731,0.2000,0.514005,884.0,742.0,0.165139,6.124596,77.517124,8.804381,211.274677,None,"LINESTRING (1603958.894 5467239.149, 1604357.4..."
147479,nzd0579-0003,nzd0579,NaN,NaN,NaN,0.200,0.1954,0.2000,0.603989,884.0,738.0,0.225269,6.270029,71.024820,8.427622,203.190705,None,"LINESTRING (1603971.439 5467333.279, 1604366.2..."


In [16]:
def create_transects(line, spacing=80, transect_length=400):
    transects = []
    distances = np.arange(0, line.length, spacing)

    for distance in distances:
        # Find point along the average line
        point = line.interpolate(distance)
        
        # Find the direction of the line at this point (tangent direction)
        nearest_point_ahead = line.interpolate(min(distance + 1e-6, line.length))
        direction = np.arctan2(nearest_point_ahead.y - point.y, nearest_point_ahead.x - point.x)
        
        # Rotate 90 degrees (perpendicular) and extend to create a transect
        transect = LineString([
            Point(
                point.x - transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y - transect_length / 2 * np.sin(direction + np.pi / 2)
            ),
            Point(
                point.x + transect_length / 2 * np.cos(direction + np.pi / 2),
                point.y + transect_length / 2 * np.sin(direction + np.pi / 2)
            )
        ])
        
        transects.append(transect)
    
    transects = gpd.GeoDataFrame(geometry=transects, crs=2193).to_crs(4326)
    return transects

all_new_transects = []
for new_shoreline in new_shorelines.to_crs(2193).itertuples():
    new_transects = create_transects(new_shoreline.geometry)
    new_transects["id"] = new_shoreline.id + "-" + new_transects.index.astype(str).str.pad(4, fillchar="0")
    new_transects["site_id"] = new_shoreline.id
    all_new_transects.append(new_transects)

all_new_transects = gpd.GeoDataFrame(pd.concat(all_new_transects).reset_index(drop=True), crs=4326)
display(all_new_transects)
m = all_new_transects.explore()
new_shorelines.explore(m=m)
new_polys.boundary.explore(m=m)
gpd.GeoSeries(all_new_transects.geometry.apply(lambda line: Point(line.coords[0])), crs=all_new_transects.crs).explore(m=m, color="red", name="transect start")
print("Make sure the origin is inland")
m

,geometry,id,site_id
0,"LINESTRING (172.69906 -43.63112, 172.69428 -43...",nzd0580-0000,nzd0580
1,"LINESTRING (172.69879 -43.63043, 172.69401 -43...",nzd0580-0001,nzd0580
2,"LINESTRING (172.69585 -43.62868, 172.69585 -43...",nzd0580-0002,nzd0580
3,"LINESTRING (172.69486 -43.62868, 172.69486 -43...",nzd0580-0003,nzd0580
4,"LINESTRING (172.69387 -43.62868, 172.69387 -43...",nzd0580-0004,nzd0580
5,"LINESTRING (172.69317 -43.62862, 172.69259 -43...",nzd0580-0005,nzd0580
6,"LINESTRING (172.69079 -43.6289, 172.69309 -43....",nzd0580-0006,nzd0580
7,"LINESTRING (172.68991 -43.62923, 172.69221 -43...",nzd0580-0007,nzd0580
8,"LINESTRING (172.6883 -43.63166, 172.69319 -43....",nzd0580-0008,nzd0580
9,"LINESTRING (172.68847 -43.63237, 172.69336 -43...",nzd0580-0009,nzd0580


Make sure the origin is inland


In [17]:
pd.concat((transects, all_new_transects)).to_file("transects_extended.geojson", driver="GeoJSON")